In [1]:
pip install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00


In [3]:
!apt-get install -y -qq glpk-utils

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 123586 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

In [1]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Define the model
model = pyo.ConcreteModel()

# Time periods (hours of the day)
T = range(24)

# Parameters
heating_demand = [1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1, 1, 1, 1, 1, 1, 1]  # in kWh
cooling_demand = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # in kWh
solar_generation = [0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0]  # in kWh
electricity_prices = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.15, 0.15, 0.15, 0.2, 0.2, 0.2, 0.2, 0.15, 0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]  # in $/kWh
battery_capacity = 10  # in kWh
battery_max_charge = 5  # in kW
battery_initial_charge = 5  # in kWh



In [2]:
# Decision variables
model.grid_usage = pyo.Var(T, domain=pyo.NonNegativeReals)
model.battery_charge = pyo.Var(T, domain=pyo.NonNegativeReals, bounds=(0, battery_max_charge))
model.battery_discharge = pyo.Var(T, domain=pyo.NonNegativeReals, bounds=(0, battery_max_charge))
model.battery_level = pyo.Var(T, domain=pyo.NonNegativeReals, bounds=(0, battery_capacity))


In [3]:

# Constraints
def energy_balance_rule(model, t):
    if t == 0:
        return model.battery_level[t] == battery_initial_charge + model.battery_charge[t] - model.battery_discharge[t]
    else:
        return model.battery_level[t] == model.battery_level[t-1] + model.battery_charge[t] - model.battery_discharge[t]

model.energy_balance = pyo.Constraint(T, rule=energy_balance_rule)

def demand_satisfaction_rule(model, t):
    return model.grid_usage[t] + solar_generation[t] + model.battery_discharge[t] - model.battery_charge[t] >= heating_demand[t] + cooling_demand[t]

model.demand_satisfaction = pyo.Constraint(T, rule=demand_satisfaction_rule)


In [4]:

# Objective function
model.cost = pyo.Objective(expr=sum(electricity_prices[t] * model.grid_usage[t] for t in T), sense=pyo.minimize)



In [5]:
# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model, tee=True)

# Display the results
for t in T:
    print(f"Hour {t}: Grid usage = {model.grid_usage[t].value:.2f} kWh, Battery charge = {model.battery_charge[t].value:.2f} kWh, Battery discharge = {model.battery_discharge[t].value:.2f} kWh, Battery level = {model.battery_level[t].value:.2f} kWh")
print(f"Total cost: {pyo.value(model.cost):.2f} $")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /tmp/tmpzb9rs8p1.glpk.raw --wglp /tmp/tmp35_iwjxx.glpk.glp --cpxlp
 /tmp/tmpqg8dj7ag.pyomo.lp
Reading problem data from '/tmp/tmpqg8dj7ag.pyomo.lp'...
48 rows, 96 columns, 167 non-zeros
440 lines were read
Writing problem data to '/tmp/tmp35_iwjxx.glpk.glp'...
435 lines were written
GLPK Simplex Optimizer 5.0
48 rows, 96 columns, 167 non-zeros
Preprocessing...
48 rows, 95 columns, 166 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 48
      0: obj =   0.000000000e+00 inf =   4.700e+01 (23)
     23: obj =   6.600000000e+00 inf =   0.000e+00 (0)
*    41: obj =   5.200000000e+00 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (116915 bytes)
Writing basic solution to '/tmp/tmpzb9rs8p1.glpk.raw'...
153 lines were written
Hour 0: Grid 